<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #6: Evaluate machine learning models</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: April 20, 2022</h5>

# Set up and requirements

### Install Python packages

In [ ]:
!git clone --recurse-submodules https://github.com/ocean-data-factory-sweden/koster_yolov4.git
!pip install -r koster_yolov4/requirements.txt

In [ ]:
# Fix libmagic upload issue
!apt-get update && apt-get install -y libmagic-dev

In [ ]:
# Solution to avoid opncv and pims library issues based on https://stackoverflow.com/questions/71204741/how-to-fix-error-module-cv2-has-no-attribute-legacy-on-python-3-7-9-and-w
!pip uninstall opencv-python -y
!pip uninstall opencv-contrib-python -y
!pip install opencv-contrib-python

### Import Python packages

In [ ]:
# Set the directory of the libraries
import sys, os
from pathlib import Path
os.chdir("koster_yolov4/notebooks")
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from ipyfilechooser import FileChooser

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.server_utils as s_utils
import kso_utils.project_utils as p_utils
import kso_utils.t3_utils as t3
import kso_utils.t4_utils as t4
import kso_utils.t5_utils as t5
import kso_utils.t8_utils as t8
from src.prepare_zooniverse import frame_aggregation
from kso_utils.zooniverse_utils import retrieve_zoo_info, populate_subjects, populate_agg_annotations

# Model-specific imports
import yolo_train as train
import yolo_test as test
import yolo_detect as detect

print("Packages loaded successfully")

### Choose your project

In [ ]:
project_name = t_utils.choose_project()

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

In [ ]:
# Specify path to store the labelled frames and annotations
fc = t_utils.choose_folder(".", "data")

### Choose your model

In [ ]:
# Store selected output path
output_folder = fc.selected

In [ ]:
# Fix important paths
data_path = [str(Path(output_folder, _)) for _ in os.listdir(output_folder) if \
             _.endswith(".yaml") and "hyp" not in _][-1]

# Choose folder that will contain the different model runs
project_path = FileChooser('/cephyr/NOBACKUP/groups/snic2021-6-9/models/koster-ml')

# Project-specific information
entity = "koster"
exp_name = "test"
display(project_path)

In [ ]:
# Choose model
eval_model = FileChooser(project_path.selected)
display(eval_model)

In [ ]:
# Find trained model weights
tuned_weights = f"{Path(project_path.selected, eval_model.selected, 'weights', 'best.pt')}"

### Evaluate model performance on test set

In [ ]:
# Evaluate YOLO Model on Unseen Test data for mAP metric

In [ ]:
test.run(data=data_path, weights=tuned_weights, conf_thres=conf_thres.value, imgsz=640)

### Run model on custom footage

In [ ]:
fc = t_utils.choose_folder(".", "custom footage")

In [ ]:
detect.run(weights=tuned_weights, 
           source=fc.selected,
           imgsz=720, conf_thres=0.7, save_txt=True, 
           project=project_path.selected)

In [ ]:
# END 